In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-tu

/content/nmt-kn-tu


In [3]:
! chmod a+x /content/nmt-kn-tu/scripts/data_preparation.sh
! chmod a+x /content/nmt-kn-tu/scripts/split_dataset.sh

In [4]:
! /content/nmt-kn-tu/scripts/data_preparation.sh data.kn data.tu

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 15.79 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 2.09 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 5.33 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads:

In [5]:
! /content/nmt-kn-tu/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.tu

In [6]:
%cd ..

/content


In [7]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [8]:
!fairseq-preprocess --source-lang kn --target-lang tu \
--trainpref nmt-kn-tu/train/train \
--validpref nmt-kn-tu/dev/dev \
--testpref nmt-kn-tu/test/test \
--destdir nmt-kn-tu/tokenized.kn-tu \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-06 04:07:05 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 04:07:06 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-tu/tokenized.kn-tu', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [9]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
!fairseq-train /content/nmt-kn-tu/tokenized.kn-tu \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "BiLSTM model - kn to tu - 6 March 2022"

2022-03-06 04:08:11 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-06 04:08:12 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 04:08:15 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'BiLSTM model - kn to tu - 6 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1

In [11]:
!fairseq-generate /content/nmt-kn-tu/tokenized.kn-tu \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-85	-2.727524995803833	<unk> ್ ಕ ್ <unk> ್ ನ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ಮಲ ್ ಜಿ
P-85	-3.8204 -0.8101 -4.7348 -0.6132 -3.3976 -2.3686 -4.7892 -1.1075 -3.3704 -2.5099 -4.7075 -1.0028 -3.3417 -2.5123 -4.6673 -1.0014 -3.3608 -2.5137 -4.6516 -0.9972 -3.3681 -2.5146 -4.6402 -0.9930 -3.3728 -4.2013 -0.0039 -3.4479 -0.2786
S-116	ತೆಗೆದು ಅದಕ ್ ಕೆ ಅಕ ್ ಕಿ ಹಾಕಿ ಕೈಮುಗಿದು ಹೋಗುತ ್ ತಾರೆ
T-116	ಕಾಯೆರ ್ ಮರತ ಏಳ ್ ಇರೆ
H-116	-2.662822961807251	ಇಂಚ <unk> ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk>
D-116	-2.662822961807251	ಇಂಚ <unk> ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk> ್ ದ ್ <unk>
P-116	-3.9475 -3.2517 -1.4406 -4.0721 -1.8446 -4.2350 -0.6754 -3.2514 -1.7412 -4.2603 -0.6715 -3.2691 -1.7485 -4.2544 -0.6675 -3.2744 -1.7551 -4.2503 -0.6636 -3.2778 -1.7597 -4.2460 -0.6596 -3.2810 -4.0725
S-213	ಇನ ್ ನು ಕಾಲದಲ ್ ಲಿ ತುಳುನಾಡ ಚರಿತ ್ ರೆಯಲ ್ ಲಿ
T-213	ನನ ಕಾಲೊಡು ತುಲುನಾಡ ಚರಿತ ್ ರೆಡೆ
H-213	-2.24354743957